<h1><b>Final Project - Data Analytics for Finance<b><h1>

<table>
      <tr><td><b>Student Name<b></td><td>Pedro Ferreira</td></tr>
      <tr><td><b>Student Number<b></td><td>38961</td></tr>
      <tr><td><b>Signal Name<b></td><td>Book leverage</td><tr>
      <tr><td><b>Signal Symbol<b></td><td>at_be</td><tr>
</table>
<p><p>

In [ ]:
// Install packages

#r "nuget: FSharp.Data"
#r "nuget: FSharp.Stats"
#r "nuget: DiffSharp-lite"
#r "nuget: Plotly.NET, 2.0.0-preview.17"

Installed Packages DiffSharp-lite, 1.0.7 FSharp.Data, 4.2.8 FSharp.Stats, 0.4.5 Plotly.NET, 2.0.0-preview.17

In [ ]:
#r "nuget: Plotly.NET.Interactive, 2.0.0-preview.17"

Installed Packages Plotly.NET.Interactive, 2.0.0-preview.17

Loading extensions from `Plotly.NET.Interactive.dll`

In [ ]:
#r "nuget: Accord"
#r "nuget: Accord.Statistics"

Installed Packages Accord, 3.8.0 Accord.Statistics, 3.8.0

In [ ]:
// Load extensions

open System
open FSharp.Data
open Plotly.NET
open FSharp.Stats
open Accord
open Accord.Statistics.Models.Regression.Linear
open DiffSharp

In [ ]:
// Set dotnet interactive formatter to plaintext
Formatter.Register(fun (x:obj) (writer: TextWriter) -> fprintfn writer "%120A" x )
Formatter.SetPreferredMimeTypesFor(typeof<obj>, "text/plain")

// Make plotly graphs work with interactive plaintext formatter
Formatter.SetPreferredMimeTypesFor(typeof<GenericChart.GenericChart>,"text/html")

In [ ]:
// Load the necessary files

let [<Literal>] ResolutionFolder = __SOURCE_DIRECTORY__
Environment.CurrentDirectory <- ResolutionFolder

#load "Common.fsx"
#load "Portfolio.fsx"
#load "YahooFinance.fsx"


In [ ]:
open Common
open Portfolio
open YahooFinance

In [ ]:
// Get the necessary data from the CSV files

let [<Literal>] IdAndReturnsFilePath = "data/id_and_return_data.csv"
let [<Literal>] MySignalFilePath = "data/at_be.csv"

In [ ]:
// Check if the first file paths are correct

IO.File.ReadLines(IdAndReturnsFilePath) |> Seq.truncate 5

seq
  ["id(string),eom(date),source(string),sizeGrp(string),obsMain(string),exchMain(string),primarySec(bool),gvkey(string),iid(string),permno(int Option),permco(int Option),excntry(string),curcd(string),fx(string),common(bool),compTpci(string),crspShrcd(int Option),compExchg(string),crsp_exchcd(int Option),adjfct(float Option),shares(float Option),gics(int Option),sic(int Option),naics(int Option),ff49(int Option),ret(float Option),retExc(float Option),prc(float Option),marketEquity(float Option)";
   "crsp_86432,2000-01-31T00:00:00.0000000,CRSP,micro,1,1,true,115876,01,86432,16313,USA,USD,1,true,,11,,3,2,5.218,40101010,6020,522110,45,-0.003906,-0.00824925,15.9375,83.161875";
   "crsp_85640,2000-01-31T00:00:00.0000000,CRSP,small,1,1,true,002193,01,85640,20300,USA,USD,1,true,,11,,1,1,102.496,35102020,8051,623110,11,-0.157143,-0.161485863,3.6875,377.954";
   "crsp_86430,2000-01-31T00:00:00.0000000,CRSP,micro,1,1,true,115946,01,86430,16319,USA,USD,1,true,,11,,3,1,10.764,45103010,7372,511

In [ ]:
// Check if the second file paths are correct

IO.File.ReadLines(MySignalFilePath) |> Seq.truncate 5

seq
  ["id(string),eom(date),signal(float option)"; "comp_001034_01,2008-12-31T00:00:00.0000000,1.5955603247";
   "comp_001043_01,2000-01-31T00:00:00.0000000,"; "comp_001076_02,2010-12-31T00:00:00.0000000,1.2305380595"; ...]


In [ ]:
// Define the CSV types

type IdAndReturnsType = 
    CsvProvider<Sample=IdAndReturnsFilePath,
                Schema="obsMain(string)->obsMain=bool,exchMain(string)->exchMain=bool",
                ResolutionFolder=ResolutionFolder>

type MySignalType = 
    CsvProvider<MySignalFilePath,
                ResolutionFolder=ResolutionFolder>

In [ ]:
// Read the data

let idAndReturnsCsv = IdAndReturnsType.GetSample()
let mySignalCsv = MySignalType.GetSample()

In [ ]:
// Convert the rows to lists

let idAndReturnsRows = idAndReturnsCsv.Rows |> Seq.toList
let mySignalRows = mySignalCsv.Rows |> Seq.toList

<h2><b>3.2 - Strategy Analysis<b><h2>

<h2>Data selection<h2>

In [ ]:
// Create a map collection from the idAndReturn data

let msfBySecurityIdAndMonth =
    idAndReturnsRows
    |> List.map(fun row -> 
        let id = Other row.Id
        let month = DateTime(row.Eom.Year,row.Eom.Month,1)
        let key = id, month
        key, row)
    |> Map

// Organize the signal data by ID and month

let signalBySecurityIdAndMonth =
    mySignalRows
    |> List.choose(fun row -> 
        match row.Signal with
        | None -> None // Remove the observations with missing signals
        | Some signal ->
            let id = Other row.Id
            let month = DateTime(row.Eom.Year,row.Eom.Month,1)
            let key = id, month
            Some (key, signal))
    |> Map 

// Group the securities by month

let securitiesByFormationMonth =
    idAndReturnsRows
    |> List.groupBy(fun x -> DateTime(x.Eom.Year, x.Eom.Month,1))
    |> List.map(fun (ym, obsThisMonth) -> 
        let idsThisMonth = [ for x in obsThisMonth do Other x.Id ]
        ym, idsThisMonth)
    |> Map

// Define the investment universe

let getInvestmentUniverse formationMonth =
    match Map.tryFind formationMonth securitiesByFormationMonth with
    | Some securities -> 
        { FormationMonth = formationMonth 
          Securities = securities }
    | None -> failwith $"{formationMonth} is not in the date range"

// Get the signal for the entire investment universe

let getMySignals (investmentUniverse: InvestmentUniverse) =
    let getMySignal (securityId, formationMonth) =
        match Map.tryFind (securityId, formationMonth) signalBySecurityIdAndMonth with
        | None -> None
        | Some signal ->
            Some { SecurityId = securityId
                   Signal = - signal } // Because a high book leverage is expected to be correlated with low returns,
    let listOfSecuritySignals =       // the signal is multiplied by -1.0 such that the top tercile has high expected returns
        investmentUniverse.Securities
        |> List.choose(fun security ->
            getMySignal (security, investmentUniverse.FormationMonth))

    { FormationMonth = investmentUniverse.FormationMonth 
      Signals = listOfSecuritySignals }
 
// Get the market cap for each security in a month

let getMarketCap (security, formationMonth) =
    match Map.tryFind (security, formationMonth) msfBySecurityIdAndMonth with
    | None -> None
    | Some row ->
        match row.MarketEquity with
        | None -> None
        | Some me -> Some (security, me)

// Get returns for each security in a month

let getSecurityReturn (security, formationMonth) =
    let missingReturn = 0.0
    match Map.tryFind (security, formationMonth) msfBySecurityIdAndMonth with
    | None -> security, missingReturn
    | Some x ->  
        match x.Ret with 
        | None -> security, missingReturn
        | Some r -> security, r

// Filters to be applied to the dataset 

let isObsMain (security, formationMonth) =
    match Map.tryFind (security, formationMonth) msfBySecurityIdAndMonth with
    | None -> false
    | Some row -> row.ObsMain

let isPrimarySecurity (security, formationMonth) =
    match Map.tryFind (security, formationMonth) msfBySecurityIdAndMonth with
    | None -> false
    | Some row -> row.PrimarySec

let isCommonStock (security, formationMonth) =
    match Map.tryFind (security, formationMonth) msfBySecurityIdAndMonth with
    | None -> false
    | Some row -> row.Common

let isExchMain (security, formationMonth) =
    match Map.tryFind (security, formationMonth) msfBySecurityIdAndMonth with
    | None -> false
    | Some row -> row.ExchMain

let hasMarketEquity (security, formationMonth) =
    match Map.tryFind (security, formationMonth) msfBySecurityIdAndMonth with
    | None -> false
    | Some row -> row.MarketEquity.IsSome

let myFilters securityAndFormationMonth =
    isObsMain securityAndFormationMonth &&
    isPrimarySecurity securityAndFormationMonth &&
    isCommonStock securityAndFormationMonth &&
    isExchMain securityAndFormationMonth &&
    isExchMain securityAndFormationMonth &&
    hasMarketEquity securityAndFormationMonth

// Filter the investment universe

let doMyFilters (universe:InvestmentUniverse) =
    let filtered =
        universe.Securities
        |> List.map(fun security -> security, universe.FormationMonth)
        |> List.filter myFilters
        |> List.map fst
    { universe with Securities = filtered}

// Define the sample months

let startSample = 
   idAndReturnsRows
   |> List.map(fun row -> DateTime(row.Eom.Year,row.Eom.Month,1))
   |> List.min

let endSample = 
    let lastMonthWithData = 
        idAndReturnsRows
        |> Seq.map(fun row -> DateTime(row.Eom.Year,row.Eom.Month,1))
        |> Seq.max
    lastMonthWithData.AddMonths(-1) 

let sampleMonths = getSampleMonths (startSample, endSample)

// Strategy function

let strategyName = "Book leverage"

let formStrategy ym =
    ym
    |> getInvestmentUniverse
    |> doMyFilters
    |> getMySignals
    |> assignSignalSort strategyName 3
    |> List.map (giveValueWeights getMarketCap)
    |> List.map (getPortfolioReturn getSecurityReturn)

// Strategy portfolios

let doParallel = true
let portfolios =
    if doParallel then
        sampleMonths
        |> List.toArray
        |> Array.Parallel.map formStrategy
        |> Array.toList
        |> List.collect id
    else
        sampleMonths
        |> List.collect formStrategy // Calculate value-weighted portfolios

// Use the French data in Common.fsx to get monthly risk-free rates

let ff3 = French.getFF3 Frequency.Monthly
let monthlyRiskFreeRate =
    [ for obs in ff3 do
        let key = DateTime(obs.Date.Year,obs.Date.Month,1)
        key, obs.Rf ]
    |> Map

// Convert the returns into excess returns

let portfolioExcessReturns =
    portfolios
    |> List.map(fun x ->
        match Map.tryFind x.YearMonth monthlyRiskFreeRate with
        | None -> failwith $"Can't find risk-free rate for {x.YearMonth}"
        | Some rf -> { x with Return = x.Return - rf })

<h2> Strategies setup<h2>

In [ ]:
// Set up the strategy that is long in the top tercile portfolio

let longOnlyStrategy =
    portfolioExcessReturns
    |> List.filter(fun x ->
        x.PortfolioId = Indexed {| Name = strategyName; Index = 3 |})

// Set up the long-short strategy that is long in the top tercile portfolio and short in the bottom tercile portfolio

let shortOnlyStrategy =
    portfolioExcessReturns
    |> List.filter(fun x ->
        x.PortfolioId = Indexed {| Name = strategyName; Index = 1 |})

let longShortStrategy =
    let shortByYearMonthMap =
        shortOnlyStrategy
        |> List.map(fun row -> row.YearMonth, row)
        |> Map
    
    [ for longObs in longOnlyStrategy do
        match Map.tryFind longObs.YearMonth shortByYearMonthMap with
        | None -> failwith "error"
        | Some shortObs ->
            { PortfolioId = Named "Long-Short Strategy"
              YearMonth = longObs.YearMonth
              Return = longObs.Return - shortObs.Return } ]

<h2>Plots<h2>

<h4><b>Cumulative return plots (without volatility management)<b><h4>

In [ ]:
// To plot the cumulative returns, we set the types to handle the value-weighted stock market
// portfolio returns and be able to compute the cumulative returns for the different portfolios

type ReturnObs =
    {
      Name: string
      Date: DateTime
      Return: float
    }

type CumulativeObs = 
    {
      Name: string
      Date: DateTime
      Growth: float
    }

let cumulativeGrowth (xs: list<ReturnObs>) =
    let sorted = xs |> List.sortBy (fun x -> x.Date)
    let calcGrowth (prior: CumulativeObs) (current: ReturnObs) =
        { Name = current.Name 
          Date = current.Date
          Growth = prior.Growth * (1.0 + current.Return) }        
    match sorted with
    | [] -> []
    | h::t ->
        let firstOb = 
            { Name = h.Name
              Date = h.Date
              Growth = 1.0 + h.Return }
        (firstOb, t) ||> List.scan calcGrowth

// Get the value-weighted stock market portfolio excess returns

let vwMktRf: list<ReturnObs> =
    let portfolioMonths =
        portfolioExcessReturns
        |> List.map(fun x -> x.YearMonth)
    let minYm = portfolioMonths |> List.min // Match the value-weighted stock market portfolio timeframe with that of the remaining portfolios
    let maxYm = portfolioMonths |> List.max

    [ for x in ff3 do
        if x.Date >= minYm && x.Date <= maxYm then
          { Name = "market"
            Date = x.Date
            Return = x.MktRf } ]

// Compute the cumulative return of the value-weighted stock market portfolio

let marketGrowth =
    vwMktRf
    |> cumulativeGrowth
    |> List.map(fun x -> x.Date, x.Growth)

// Compute the cumulative return of the long-only strategy portfolio

let longOnlyGrowth =
    longOnlyStrategy
    |> List.map(fun x -> // Convert the long-only portfolio return data to a type compatible with cumulativeGrowth (ReturnObs)
        {
          Name = "Long-only"
          Date = x.YearMonth
          Return = x.Return
        } )
    |> cumulativeGrowth
    |> List.map(fun x -> x.Date, x.Growth)

// Compute the cumulative return of the long-short strategy portfolio

let longShortGrowth =
    longShortStrategy
    |> List.map(fun x -> // Convert the long-short portfolio return data to a type compatible with cumulativeGrowth (ReturnObs)
        {
          Name = "Long-short"
          Date = x.YearMonth
          Return = x.Return
        })
    |> cumulativeGrowth
    |> List.map(fun x -> x.Date, x.Growth)

// Cumulative returns (long-only, long-short and value-weighted stock market portfolios)

Chart.combine(
    [ Chart.Line(
        marketGrowth,
        Name="Market portfolio")
      Chart.Line(
        longOnlyGrowth,
        Name="Long-only portfolio")
      Chart.Line(
        longShortGrowth,
        Name="Long-short portfolio")
    |> Chart.withTitle("Cumulative returns (without volatility management)")
    ]
)

<!-- Plotly chart will be drawn inside this DIV -->

<h4><b>Cumulative return plots (target annualized volatility of 10%)<b><h4>

In [ ]:
// Function to annualize the monthly standard deviation for each portfolio

let annualizeMonthlyStdDev monthlyStdDev = sqrt(12.0) * monthlyStdDev


// Value-weighted stock market portfolio standard deviation

let stdDevMkt =
    vwMktRf
    |> Seq.stDevBy(fun x -> x.Return)
    |> annualizeMonthlyStdDev

// Long-only portfolio standard deviation

let stdLongOnly =
    longOnlyStrategy
    |> Seq.stDevBy(fun x -> x.Return)
    |> annualizeMonthlyStdDev

// Long-short portfolio standard deviation

let stdLongShort =
    longShortStrategy
    |> Seq.stDevBy(fun x -> x.Return)
    |> annualizeMonthlyStdDev

// Value-weighted stock market portfolio levered to have an annualized volatility of 10%

let vwMktRf10 =
    vwMktRf
    |> List.map(fun x ->
        let weightVwMktRf10 = (0.10/stdDevMkt)
        {
            Name = "Mkt 10"
            Date = x.Date
            Return = x.Return
        })

// Long-only portfolio levered to have an annualized volatility of 10%

let longOnly10 =
    longOnlyStrategy
    |> List.map(fun x ->
        let weightLongOnly10 = (0.10/stdLongOnly)
        {
            Name = "Long-only 10"
            Date = x.YearMonth
            Return = x.Return * weightLongOnly10
        })

// Long-short portfolio levered to have an annualized volatility of 10%

let longShort10 =
    longShortStrategy
    |> List.map(fun x ->
        let weightLongShort10 = (0.10/stdLongShort)
        {
            Name = "Long-short 10"
            Date = x.YearMonth
            Return = x.Return * weightLongShort10
        })

// Compute the cumulative return of the value-weighted stock market portfolio

let market10Growth =
    vwMktRf10
    |> cumulativeGrowth
    |> List.map(fun x -> x.Date, x.Growth)

// Compute the cumulative return of the long-only strategy portfolio

let longOnly10Growth =
    longOnly10
    |> List.map(fun x -> // Convert the long-only portfolio return data to a type compatible with cumulativeGrowth (ReturnObs)
        {
          Name = "Long-only"
          Date = x.Date
          Return = x.Return
        } )
    |> cumulativeGrowth
    |> List.map(fun x -> x.Date, x.Growth)

// Compute the cumulative return of the long-short strategy portfolio

let longShort10Growth =
    longShort10
    |> List.map(fun x -> // Convert the long-short portfolio return data to a type compatible with cumulativeGrowth (ReturnObs)
        {
          Name = "Long-short"
          Date = x.Date
          Return = x.Return
        })
    |> cumulativeGrowth
    |> List.map(fun x -> x.Date, x.Growth)

// Cumulative returns (long-only, long-short and value-weighted stock market portfolios)

Chart.combine(
    [ Chart.Line(
        market10Growth,
        Name="Market portfolio")
      Chart.Line(
        longOnly10Growth,
        Name="Long-only portfolio")
      Chart.Line(
        longShort10Growth,
        Name="Long-short portfolio")
    |> Chart.withTitle("Cumulative returns (volatility-managed portfolios)")
    ]
)


<!-- Plotly chart will be drawn inside this DIV -->

<h2>Average annual excess return and Sharpe ratio (without volatility management)<h2>

<h4><b>Long-only and long-short portfolios (full period)<b><h4>

In [ ]:
// AVERAGE ANNUALIZED EXCESS RETURNS

let annualizeMonthlyRet x = (1.0 + x) ** 12.0 - 1.0

// Average annualized exccess return for the long-only portfolio

let longOnlyAvg =
    longOnlyStrategy
    |> List.averageBy(fun x -> x.Return)
    |> annualizeMonthlyRet

// Average annualized excess return for the long-short portfolio

let longShortAvg =
    longShortStrategy
    |> List.averageBy(fun x -> x.Return)
    |> annualizeMonthlyRet


// ANNUALIZED SHARPE RATIOS

let annualizeMonthlySharpe monthlySharpe = sqrt(12.0) * monthlySharpe

let sharpe (xs: float seq) =
    (Seq.mean xs) / (Seq.stDev xs)

// Annualized Sharpe ratio for the long-only portfolio

let sharpeLongOnly =
    longOnlyStrategy
    |> List.map(fun x -> x.Return)
    |> sharpe
    |> annualizeMonthlySharpe

// Annualized Sharpe ratio for the long-short portfolio

let sharpeLongShort =
    longShortStrategy
    |> List.map(fun x -> x.Return)
    |> sharpe
    |> annualizeMonthlySharpe

<h4><b>Long-only and long-short portfolios (first half of the sample)<b><h4>

In [ ]:
// AVERAGE ANNUALIZED EXCESS RETURNS

// Average annualized exccess return for the long-only portfolio

let longOnlyAvg1stHalf =
    let firstHalf =
        longOnlyStrategy
        |> List.splitInto 2
        |> List.min
    let annualRet =
        firstHalf
        |> List.averageBy(fun x -> x.Return)
        |> annualizeMonthlyRet
    annualRet

// Average annualized excess return for the long-short portfolio

let longShortAvg1stHalf =
    let firstHalf =
        longShortStrategy
        |> List.splitInto 2
        |> List.min
    let annualRet =
        firstHalf
        |> List.averageBy(fun x -> x.Return)
        |> annualizeMonthlyRet
    annualRet


// ANNUALIZED SHARPE RATIOS

// Annualized Sharpe ratio for the long-only portfolio

let sharpeLongOnly1stHalf =
    let firstHalf =
        longOnlyStrategy
        |> List.splitInto 2
        |> List.min
    let annualSharpe =
        firstHalf
        |> List.map(fun x -> x.Return)
        |> sharpe
        |> annualizeMonthlySharpe
    annualSharpe

// Annualized Sharpe ratio for the long-short portfolio

let sharpeLongShort1stHalf =
    let firstHalf =
        longShortStrategy
        |> List.splitInto 2
        |> List.min
    let annualSharpe =
        firstHalf
        |> List.map(fun x -> x.Return)
        |> sharpe
        |> annualizeMonthlySharpe
    annualSharpe

<h4><b>Long-only and long-short portfolios (second half of the sampe)<b><h4>

In [ ]:
// AVERAGE ANNUALIZED EXCESS RETURNS

// Average annualized exccess return for the long-only portfolio

let longOnlyAvg2ndHalf =
    let secondHalf =
        longOnlyStrategy
        |> List.splitInto 2
        |> List.max
    let annualRet =
        secondHalf
        |> List.averageBy(fun x -> x.Return)
        |> annualizeMonthlyRet
    annualRet

// Average annualized excess return for the long-short portfolio

let longShortAvg2ndHalf =
    let secondHalf =
        longShortStrategy
        |> List.splitInto 2
        |> List.max
    let annualRet =
        secondHalf
        |> List.averageBy(fun x -> x.Return)
        |> annualizeMonthlyRet
    annualRet


// ANNUALIZED SHARPE RATIOS

// Annualized Sharpe ratio for the long-only portfolio

let sharpeLongOnly2ndHalf =
    let firstHalf =
        longOnlyStrategy
        |> List.splitInto 2
        |> List.max
    let annualSharpe =
        firstHalf
        |> List.map(fun x -> x.Return)
        |> sharpe
        |> annualizeMonthlySharpe
    annualSharpe

// Annualized Sharpe ratio for the long-short portfolio

let sharpeLongShort2ndHalf =
    let firstHalf =
        longShortStrategy
        |> List.splitInto 2
        |> List.max
    let annualSharpe =
        firstHalf
        |> List.map(fun x -> x.Return)
        |> sharpe
        |> annualizeMonthlySharpe
    annualSharpe

<h4>Value-weighted stock market portfolio (full period, first half and second half periods)

In [ ]:
// AVERAGE ANNUALIZED EXCESS 

// Average annualized exccess return (full period)

let vwMktRfAvgFull =
    vwMktRf
    |> List.averageBy(fun x -> x.Return)
    |> annualizeMonthlyRet

// Average annualized exccess return (first half)

let vwMktRfAvg1stHalf =
    let firstHalf =
        vwMktRf
        |> List.splitInto 2
        |> List.min
    let annualRet =
        firstHalf
        |> List.averageBy(fun x -> x.Return)
        |> annualizeMonthlyRet
    annualRet

// Average annualized excess return (second half)

let vwMktRfAvg2ndHalf =
    let secondHalf =
        vwMktRf
        |> List.splitInto 2
        |> List.max
    let annualRet =
        secondHalf
        |> List.averageBy(fun x -> x.Return)
        |> annualizeMonthlyRet
    annualRet


// ANNUALIZED SHARPE RATIOS

// Annualized Sharpe ratio (full period)

let sharpeVwMktRf =
    vwMktRf
    |> List.map(fun x -> x.Return)
    |> sharpe
    |> annualizeMonthlySharpe

// Annualized Sharpe ratio (first half)

let sharpeVwMktRf1stHalf =
    let firstHalf =
        vwMktRf
        |> List.splitInto 2
        |> List.min
    let annualSharpe =
        firstHalf
        |> List.map(fun x -> x.Return)
        |> sharpe
        |> annualizeMonthlySharpe
    annualSharpe

// Annualized Sharpe ratio (second half)

let sharpeVwMktRf2ndHalf =
    let secondHalf =
        vwMktRf
        |> List.splitInto 2
        |> List.max
    let annualSharpe =
        secondHalf
        |> List.map(fun x -> x.Return)
        |> sharpe
        |> annualizeMonthlySharpe
    annualSharpe

<h2>CAPM and FF3 analysis<h2>

In [ ]:
// Set up the OLS trainer

type RegData =
    { Date: DateTime
      Portfolio: float
      MktRf: float
      Hml: float
      Smb: float
    }

type RegressionOutput =
  { Model: MultipleLinearRegression
    TValuesWeights: float array
    TValuesIntercept: float
    R2: float
  }

type Prediction =
  { Label : float
    Score: float
  }


let ff3ByMonth =
    ff3
    |> Array.map (fun x -> DateTime(x.Date.Year, x.Date.Month,1), x)
    |> Map

let fitModel (x: (float array) array, y: float array) =
    let ols = new OrdinaryLeastSquares (UseIntercept=true)
    let estimate = ols.Learn(x,y)
    let mse = estimate.GetStandardError(x,y)
    let se = estimate.GetStandardErrors(mse, ols.GetInformationMatrix())
    let tvaluesWeights =
        estimate.Weights
        |> Array.mapi (fun i w -> w / se.[i])
    let tvalueIntercept = estimate.Intercept / (se |> Array.last)
    let r2 = estimate.CoefficientOfDetermination(x,y)
    { Model = estimate
      TValuesWeights = tvaluesWeights
      TValuesIntercept = tvalueIntercept
      R2 = r2} 

// To be able to compute the information ratios

let makePredictions
    (estimate:MultipleLinearRegression)
    (x: (float array) array, y: float array) =
    (estimate.Transform(x), y)
    ||> Array.zip
    |> Array.map (fun (score, label) -> { Score = score; Label = label })

<h4><b>Full period regression data (long-only and long-short portfolios)<b><h4>

In [ ]:
// LONG-ONLY PORTFOLIO

let regDataLongOnlyFull =
    longOnlyStrategy
    |> List.toArray
    |> Array.map (fun port ->
        let monthToFind = DateTime(port.YearMonth.Year, port.YearMonth.Month,1)
        match Map.tryFind monthToFind ff3ByMonth with
        | None -> failwith "error"
        | Some ff3 ->
            { Date = monthToFind
              Portfolio = port.Return // longShort includes excess returns; no need to subtract the risk-free rate
              MktRf = ff3.MktRf
              Hml = ff3.Hml
              Smb = ff3.Smb })

// CAPM estimation (annualized alpha, t-stat and information ratio)

let capmModelDataLongOnlyFull =
    regDataLongOnlyFull
    |> Array.map (fun obs -> [|obs.MktRf|], obs.Portfolio)
    |> Array.unzip

let capmEstimateLongOnlyFull = capmModelDataLongOnlyFull |> fitModel

let capmAnnualAlphaLongOnlyFull = 12.0 * capmEstimateLongOnlyFull.Model.Intercept

let residualsCapmLOFull (xs: Prediction array) = xs |> Array.map(fun x -> x.Label - x.Score)
let capmPredictionsLOFull = makePredictions capmEstimateLongOnlyFull.Model capmModelDataLongOnlyFull
let capmResidualsLOFull = residualsCapmLOFull capmPredictionsLOFull
let capmStDevResidualsLOFull = sqrt(12.0) * (Seq.stDev capmResidualsLOFull)
let capmInformationRatioLOFull = capmAnnualAlphaLongOnlyFull / capmStDevResidualsLOFull

// FF3 estimation (annualized alpha and t-stat)

let ff3ModelDataLongOnlyFull =
    regDataLongOnlyFull
    |> Array.map (fun obs -> [|obs.MktRf; obs.Hml; obs.Smb|], obs.Portfolio)
    |> Array.unzip

let ff3EstimateLongOnlyFull = ff3ModelDataLongOnlyFull |> fitModel

let ff3AnnualAlphaLongOnlyFull = 12.0 * ff3EstimateLongOnlyFull.Model.Intercept
ff3AnnualAlphaLongOnlyFull, ff3EstimateLongOnlyFull.TValuesIntercept

let residualsff3LOFull (xs: Prediction array) = xs |> Array.map(fun x -> x.Label - x.Score)
let ff3PredictionsLOFull = makePredictions ff3EstimateLongOnlyFull.Model ff3ModelDataLongOnlyFull
let ff3ResidualsLOFull = residualsff3LOFull ff3PredictionsLOFull
let ff3StDevResidualsLOFull = sqrt(12.0) * (Seq.stDev ff3ResidualsLOFull)
let ff3InformationRatioLOFull = ff3AnnualAlphaLongOnlyFull / ff3StDevResidualsLOFull



// LONG-SHORT PORTFOLIO

let regDataLongShortFull =
    longShortStrategy
    |> List.toArray
    |> Array.map (fun port ->
        let monthToFind = DateTime(port.YearMonth.Year, port.YearMonth.Month,1)
        match Map.tryFind monthToFind ff3ByMonth with
        | None -> failwith "error"
        | Some ff3 ->
            { Date = monthToFind
              Portfolio = port.Return // longShort includes excess returns; no need to subtract the risk-free rate
              MktRf = ff3.MktRf
              Hml = ff3.Hml
              Smb = ff3.Smb })

// CAPM estimation (annualized alpha, t-stat and information ratio)

let capmModelDataLongShortFull =
    regDataLongShortFull
    |> Array.map (fun obs -> [|obs.MktRf|], obs.Portfolio)
    |> Array.unzip

let capmEstimateLongShortFull = capmModelDataLongShortFull |> fitModel

let capmAnnualAlphaLongShortFull = 12.0 * capmEstimateLongShortFull.Model.Intercept

let residualsCapmLSFull (xs: Prediction array) = xs |> Array.map(fun x -> x.Label - x.Score)
let capmPredictionsLSFull = makePredictions capmEstimateLongShortFull.Model capmModelDataLongShortFull
let capmResidualsLSFull = residualsCapmLSFull capmPredictionsLSFull
let capmStDevResidualsLSFull = sqrt(12.0) * (Seq.stDev capmResidualsLSFull)
let capmInformationRatioLSFull = capmAnnualAlphaLongShortFull / capmStDevResidualsLSFull

// FF3 estimation (annualized alpha and t-stat)

let ff3ModelDataLongShortFull =
    regDataLongShortFull
    |> Array.map (fun obs -> [|obs.MktRf; obs.Hml; obs.Smb|], obs.Portfolio)
    |> Array.unzip

let ff3EstimateLongShortFull = ff3ModelDataLongShortFull |> fitModel

let ff3AnnualAlphaLongShortFull = 12.0 * ff3EstimateLongShortFull.Model.Intercept

let residualsff3LSFull (xs: Prediction array) = xs |> Array.map(fun x -> x.Label - x.Score)
let ff3PredictionsLSFull = makePredictions ff3EstimateLongShortFull.Model ff3ModelDataLongShortFull
let ff3ResidualsLSFull = residualsff3LSFull ff3PredictionsLSFull
let ff3StDevResidualsLSFull = sqrt(12.0) * (Seq.stDev ff3ResidualsLSFull)
let ff3InformationRatioLSFull = ff3AnnualAlphaLongShortFull / ff3StDevResidualsLSFull

<h4><b>First half regression data (long-only and long-short portfolios)<h4><b>

In [ ]:
// LONG-ONLY PORTFOLIO

let regDataLongOnly1 =
    let firstHalf =
        longOnlyStrategy
        |> List.splitInto 2
        |> List.min
    firstHalf
    |> List.toArray
    |> Array.map (fun port ->
        let monthToFind = DateTime(port.YearMonth.Year, port.YearMonth.Month,1)
        match Map.tryFind monthToFind ff3ByMonth with
        | None -> failwith "error"
        | Some ff3 ->
            { Date = monthToFind
              Portfolio = port.Return // longShort includes excess returns; no need to subtract the risk-free rate
              MktRf = ff3.MktRf
              Hml = ff3.Hml
              Smb = ff3.Smb })

// CAPM estimation (annualized alpha, t-stat and information ratio)

let capmModelDataLongOnly1 =
    regDataLongOnly1
    |> Array.map (fun obs -> [|obs.MktRf|], obs.Portfolio)
    |> Array.unzip

let capmEstimateLongOnly1 = capmModelDataLongOnly1 |> fitModel

let capmAnnualAlphaLongOnly1 = 12.0 * capmEstimateLongOnly1.Model.Intercept

let residualsCapmLO1 (xs: Prediction array) = xs |> Array.map(fun x -> x.Label - x.Score)
let capmPredictionsLO1 = makePredictions capmEstimateLongOnly1.Model capmModelDataLongOnly1
let capmResidualsLO1 = residualsCapmLO1 capmPredictionsLO1
let capmStDevResidualsLO1 = sqrt(12.0) * (Seq.stDev capmResidualsLO1)
let capmInformationRatioLO1 = capmAnnualAlphaLongOnly1 / capmStDevResidualsLO1

// FF3 estimation (annualized alpha and t-stat)

let ff3ModelDataLongOnly1 =
    regDataLongOnly1
    |> Array.map (fun obs -> [|obs.MktRf; obs.Hml; obs.Smb|], obs.Portfolio)
    |> Array.unzip

let ff3EstimateLongOnly1 = ff3ModelDataLongOnly1 |> fitModel

let ff3AnnualAlphaLongOnly1 = 12.0 * ff3EstimateLongOnly1.Model.Intercept

let residualsff3LO1 (xs: Prediction array) = xs |> Array.map(fun x -> x.Label - x.Score)
let ff3PredictionsLO1 = makePredictions ff3EstimateLongOnly1.Model ff3ModelDataLongOnly1
let ff3ResidualsLO1 = residualsff3LO1 ff3PredictionsLO1
let ff3StDevResidualsLO1 = sqrt(12.0) * (Seq.stDev ff3ResidualsLO1)
let ff3InformationRatioLO1 = ff3AnnualAlphaLongOnly1 / ff3StDevResidualsLO1



// LONG-SHORT PORTFOLIO

let regDataLongShort1 =
    let firstHalf =
        longShortStrategy
        |> List.splitInto 2
        |> List.min
    firstHalf
    |> List.toArray
    |> Array.map (fun port ->
        let monthToFind = DateTime(port.YearMonth.Year, port.YearMonth.Month,1)
        match Map.tryFind monthToFind ff3ByMonth with
        | None -> failwith "error"
        | Some ff3 ->
            { Date = monthToFind
              Portfolio = port.Return // longShort includes excess returns; no need to subtract the risk-free rate
              MktRf = ff3.MktRf
              Hml = ff3.Hml
              Smb = ff3.Smb })

// CAPM estimation (annualized alpha, t-stat and information ratio)

let capmModelDataLongShort1 =
    regDataLongShort1
    |> Array.map (fun obs -> [|obs.MktRf|], obs.Portfolio)
    |> Array.unzip

let capmEstimateLongShort1 = capmModelDataLongShort1 |> fitModel

let capmAnnualAlphaLongShort1 = 12.0 * capmEstimateLongShort1.Model.Intercept

let residualsCapmLS1 (xs: Prediction array) = xs |> Array.map(fun x -> x.Label - x.Score)
let capmPredictionsLS1 = makePredictions capmEstimateLongShort1.Model capmModelDataLongShort1
let capmResidualsLS1 = residualsCapmLS1 capmPredictionsLS1
let capmStDevResidualsLS1 = sqrt(12.0) * (Seq.stDev capmResidualsLS1)
let capmInformationRatioLS1 = capmAnnualAlphaLongShort1 / capmStDevResidualsLS1

// FF3 estimation (annualized alpha and t-stat)

let ff3ModelDataLongShort1 =
    regDataLongShort1
    |> Array.map (fun obs -> [|obs.MktRf; obs.Hml; obs.Smb|], obs.Portfolio)
    |> Array.unzip

let ff3EstimateLongShort1 = ff3ModelDataLongShort1 |> fitModel

let ff3AnnualAlphaLongShort1 = 12.0 * ff3EstimateLongShort1.Model.Intercept

let residualsff3LS1 (xs: Prediction array) = xs |> Array.map(fun x -> x.Label - x.Score)
let ff3PredictionsLS1 = makePredictions ff3EstimateLongShort1.Model ff3ModelDataLongShort1
let ff3ResidualsLS1 = residualsff3LS1 ff3PredictionsLS1
let ff3StDevResidualsLS1 = sqrt(12.0) * (Seq.stDev ff3ResidualsLS1)
let ff3InformationRatioLS1 = ff3AnnualAlphaLongShort1 / ff3StDevResidualsLS1

<h4><b>Second half regression data (long-only and long-short portfolios)<h4><b>

In [ ]:
// LONG-ONLY PORTFOLIO

let regDataLongOnly2 =
    let secondHalf =
        longOnlyStrategy
        |> List.splitInto 2
        |> List.max
    secondHalf
    |> List.toArray
    |> Array.map (fun port ->
        let monthToFind = DateTime(port.YearMonth.Year, port.YearMonth.Month,1)
        match Map.tryFind monthToFind ff3ByMonth with
        | None -> failwith "error"
        | Some ff3 ->
            { Date = monthToFind
              Portfolio = port.Return // longShort includes excess returns; no need to subtract the risk-free rate
              MktRf = ff3.MktRf
              Hml = ff3.Hml
              Smb = ff3.Smb })

// CAPM estimation (annualized alpha, t-stat and information ratio)

let capmModelDataLongOnly2 =
    regDataLongOnly2
    |> Array.map (fun obs -> [|obs.MktRf|], obs.Portfolio)
    |> Array.unzip

let capmEstimateLongOnly2 = capmModelDataLongOnly2 |> fitModel

let capmAnnualAlphaLongOnly2 = 12.0 * capmEstimateLongOnly2.Model.Intercept
capmAnnualAlphaLongOnly2, capmEstimateLongOnly2.TValuesIntercept

let residualsCapmLO2 (xs: Prediction array) = xs |> Array.map(fun x -> x.Label - x.Score)
let capmPredictionsLO2 = makePredictions capmEstimateLongOnly2.Model capmModelDataLongOnly2
let capmResidualsLO2 = residualsCapmLO2 capmPredictionsLO2
let capmStDevResidualsLO2 = sqrt(12.0) * (Seq.stDev capmResidualsLO2)
let capmInformationRatioLO2 = capmAnnualAlphaLongOnly2 / capmStDevResidualsLO2

// FF3 estimation (annualized alpha and t-stat)

let ff3ModelDataLongOnly2 =
    regDataLongOnly2
    |> Array.map (fun obs -> [|obs.MktRf; obs.Hml; obs.Smb|], obs.Portfolio)
    |> Array.unzip

let ff3EstimateLongOnly2 = ff3ModelDataLongOnly2 |> fitModel

let ff3AnnualAlphaLongOnly2 = 12.0 * ff3EstimateLongOnly2.Model.Intercept
ff3AnnualAlphaLongOnly2, ff3EstimateLongOnly2.TValuesIntercept

let residualsff3LO2 (xs: Prediction array) = xs |> Array.map(fun x -> x.Label - x.Score)
let ff3PredictionsLO2 = makePredictions ff3EstimateLongOnly2.Model ff3ModelDataLongOnly2
let ff3ResidualsLO2 = residualsff3LO2 ff3PredictionsLO2
let ff3StDevResidualsLO2 = sqrt(12.0) * (Seq.stDev ff3ResidualsLO2)
let ff3InformationRatioLO2 = ff3AnnualAlphaLongOnly2 / ff3StDevResidualsLO2



// LONG-SHORT PORTFOLIO

let regDataLongShort2 =
    let secondHalf =
        longShortStrategy
        |> List.splitInto 2
        |> List.max
    secondHalf
    |> List.toArray
    |> Array.map (fun port ->
        let monthToFind = DateTime(port.YearMonth.Year, port.YearMonth.Month,1)
        match Map.tryFind monthToFind ff3ByMonth with
        | None -> failwith "error"
        | Some ff3 ->
            { Date = monthToFind
              Portfolio = port.Return // longShort includes excess returns; no need to subtract the risk-free rate
              MktRf = ff3.MktRf
              Hml = ff3.Hml
              Smb = ff3.Smb })

// CAPM estimation (annualized alpha, t-stat and information ratio)

let capmModelDataLongShort2 =
    regDataLongShort2
    |> Array.map (fun obs -> [|obs.MktRf|], obs.Portfolio)
    |> Array.unzip

let capmEstimateLongShort2 = capmModelDataLongShort2 |> fitModel

let capmAnnualAlphaLongShort2 = 12.0 * capmEstimateLongShort2.Model.Intercept
capmAnnualAlphaLongShort2, capmEstimateLongShort2.TValuesIntercept

let residualsCapmLS2 (xs: Prediction array) = xs |> Array.map(fun x -> x.Label - x.Score)
let capmPredictionsLS2 = makePredictions capmEstimateLongShort2.Model capmModelDataLongShort2
let capmResidualsLS2 = residualsCapmLS2 capmPredictionsLS2
let capmStDevResidualsLS2 = sqrt(12.0) * (Seq.stDev capmResidualsLS2)
let capmInformationRatioLS2 = capmAnnualAlphaLongShort2 / capmStDevResidualsLS2

// FF3 estimation (annualized alpha and t-stat)

let ff3ModelDataLongShort2 =
    regDataLongShort2
    |> Array.map (fun obs -> [|obs.MktRf; obs.Hml; obs.Smb|], obs.Portfolio)
    |> Array.unzip

let ff3EstimateLongShort2 = ff3ModelDataLongShort2 |> fitModel

let ff3AnnualAlphaLongShort2 = 12.0 * ff3EstimateLongShort2.Model.Intercept
ff3AnnualAlphaLongShort2, ff3EstimateLongShort2.TValuesIntercept

let residualsff3LS2 (xs: Prediction array) = xs |> Array.map(fun x -> x.Label - x.Score)
let ff3PredictionsLS2 = makePredictions ff3EstimateLongShort2.Model ff3ModelDataLongShort2
let ff3ResidualsLS2 = residualsff3LS2 ff3PredictionsLS2
let ff3StDevResidualsLS2 = sqrt(12.0) * (Seq.stDev ff3ResidualsLS2)
let ff3InformationRatioLS2 = ff3AnnualAlphaLongShort2 / ff3StDevResidualsLS2


<h4><b>Summary<b><h4>

In [ ]:
let header1 = ["<b>Portfolio<b>";"<b>Average Annualized Excess Return (%)<b>";"<b>Annualized Sharpe Ratio<b>";"<b>CAPM Alpha (%)<b>";"<b>CAPM t-statistic<b>";"<b>CAPM Information Ratio<b>";"<b>FF3 Alpha (%)<b>";"<b>FF3 t-statistic<b>";"<b>FF3 Information Ratio<b>"]
let rows1 =
    [
        ["Long-only Full Period";(round 2 (longOnlyAvg * 100.0)).ToString();(round 2 sharpeLongOnly).ToString();(round 2 (capmAnnualAlphaLongOnlyFull * 100.0)).ToString();(round 2 capmEstimateLongOnlyFull.TValuesIntercept).ToString();(round 2 capmInformationRatioLOFull).ToString();(round 2 (ff3AnnualAlphaLongOnlyFull * 100.0)).ToString();(round 2 ff3EstimateLongOnlyFull.TValuesIntercept).ToString();(round 2 ff3InformationRatioLOFull).ToString()]
        ["Long-only First Half";(round 3 (longOnlyAvg1stHalf * 100.0)).ToString();(round 2 sharpeLongOnly1stHalf).ToString();(round 2 (capmAnnualAlphaLongOnly1 * 100.0)).ToString();(round 2 capmEstimateLongOnly1.TValuesIntercept).ToString();(round 2 capmInformationRatioLO1).ToString();(round 2 (ff3AnnualAlphaLongOnly1 * 100.0)).ToString();(round 2 ff3EstimateLongOnly1.TValuesIntercept).ToString();(round 3 ff3InformationRatioLO1).ToString()]
        ["Long-only Second Half";(round 3 (longOnlyAvg2ndHalf * 100.0)).ToString();(round 2 sharpeLongOnly2ndHalf).ToString();(round 3 (capmAnnualAlphaLongOnly2 * 100.0)).ToString();(round 2 capmEstimateLongOnly2.TValuesIntercept).ToString();(round 2 capmInformationRatioLO2).ToString();(round 2 (ff3AnnualAlphaLongOnly2 * 100.0)).ToString();(round 2 ff3EstimateLongOnly2.TValuesIntercept).ToString();(round 2 ff3InformationRatioLO2).ToString()]
        ["Long-short Full Period";(round 2 (longShortAvg * 100.0)).ToString();(round 2 sharpeLongShort).ToString();(round 2 (capmAnnualAlphaLongShortFull * 100.0)).ToString();(round 2 capmEstimateLongShortFull.TValuesIntercept).ToString();(round 2 capmInformationRatioLSFull).ToString();(round 2 (ff3AnnualAlphaLongShortFull * 100.0)).ToString();(round 2 ff3EstimateLongShortFull.TValuesIntercept).ToString();(round 2 ff3InformationRatioLSFull).ToString()]
        ["Long-short First Half";(round 2 (longShortAvg1stHalf * 100.0)).ToString();(round 2 sharpeLongShort1stHalf).ToString();(round 2 (capmAnnualAlphaLongShort1 * 100.0)).ToString();(round 2 capmEstimateLongShort1.TValuesIntercept).ToString();(round 2 capmInformationRatioLS1).ToString();(round 2 (ff3AnnualAlphaLongShort1 * 100.0)).ToString();(round 2 ff3EstimateLongShort1.TValuesIntercept).ToString();(round 2 ff3InformationRatioLS1).ToString()]
        ["Long-short Second Half";(round 2 (longShortAvg2ndHalf * 100.0)).ToString();(round 2 sharpeLongShort2ndHalf).ToString();(round 2 (capmAnnualAlphaLongShort2 * 100.0)).ToString();(round 2 capmEstimateLongShort2.TValuesIntercept).ToString();(round 2 capmInformationRatioLS2).ToString();(round 2 (ff3AnnualAlphaLongShort2 * 100.0)).ToString();(round 2 ff3EstimateLongShort2.TValuesIntercept).ToString();(round 2 ff3InformationRatioLS2).ToString()]
        ["VW MktRf Full Period";(round 2 (vwMktRfAvgFull * 100.0)).ToString();(round 2 sharpeVwMktRf).ToString();"";"";"";"";"";""]
        ["VW MktRf First Half";(round 2 (vwMktRfAvg1stHalf * 100.0)).ToString();(round 2 sharpeVwMktRf1stHalf).ToString();"";"";"";"";"";""]
        ["VW MktRf Second Half";(round 2 (vwMktRfAvg2ndHalf * 100.0)).ToString();(round 2 sharpeVwMktRf2ndHalf).ToString();"";"";"";"";"";""]
    ]

let table1 = Chart.Table(header1, rows1) |> Chart.withSize (2000.0, 500.0) |> Chart.withTitle("Key Statistics Summary")
table1

<!-- Plotly chart will be drawn inside this DIV -->

<h2><b>3.3 - Strategy as part of a diversified portfolio<b><h2>

In [ ]:
// Define the type to hold the data from Yahoo Finance

type StockData =
    { Symbol : string 
      Date : DateTime
      Return : float }

// Tickers for Vanguard Total Stock Market ETF (VTI) and Vanguard Total Bond Market ETF (BND)

let tickers = 
    [ 
        "VTI" 
        "BND" 
    ]

// Get the prices for the ETFs above from Yahoo Finance

let tickPrices = 
    YahooFinance.PriceHistory(
        tickers,
        startDate = startSample, // Set to coincide with the first month in which we can form the long-only and long-short portfolios
        interval = Monthly)

// Compute montlhy returns from the price observations to type StockData

let pricesToReturns (symbol, adjPrices: list<PriceObs>) =
    adjPrices
    |> List.sortBy (fun x -> x.Date)
    |> List.pairwise
    |> List.map (fun (day0, day1) ->
        let r = day1.AdjustedClose / day0.AdjustedClose - 1.0 
        { Symbol = symbol 
          Date = day1.Date 
          Return = r })
  
let tickReturns =
    tickPrices
    |> List.groupBy (fun x -> x.Symbol)
    |> List.collect pricesToReturns

// Convert the long-only and long-short data to type StockData

let longOnlyPortfolio = 
    longOnlyStrategy
    |> List.map(fun i ->
        { Symbol = "Book leverage"
          Date = i.YearMonth
          Return = i.Return })

let longShortPortfolio = 
    longShortStrategy
    |> List.map(fun i ->
        { Symbol = "Book leverage"
          Date = i.YearMonth
          Return = i.Return })  

// To compute excess returns, we must transform the FF3 data to a StockData record type

let ff3new = ff3 |> Array.toList

let ff3StockData =
    [ 
       ff3new |> List.map(fun x -> {Symbol="HML";Date=x.Date;Return=x.Hml})
       ff3new |> List.map(fun x -> {Symbol="MktRf";Date=x.Date;Return=x.MktRf})
       ff3new |> List.map(fun x -> {Symbol="Smb";Date=x.Date;Return=x.Smb})
    ] |> List.concat

// Then convert to excess returns

let rf = Map [ for x in ff3new do x.Date, x.Rf ]

let standardInvestmentsExcess =
    let maxff3Date = ff3new |> List.map(fun x -> x.Date) |> List.max
    tickReturns
    |> List.filter(fun x -> x.Date <= maxff3Date)
    |> List.map(fun x -> 
        match Map.tryFind x.Date rf with 
        | None -> failwith $"why isn't there a rf for {x.Date}"
        | Some rf -> { x with Return = x.Return - rf })

// Put the stocks in a map keyed by symbol

let stockData portfolio =
    List.concat [|standardInvestmentsExcess; portfolio|]
    |> List.groupBy(fun x -> x.Symbol)
    |> Map

let stockData6040 =
    standardInvestmentsExcess
    |> List.groupBy(fun x -> x.Symbol)
    |> Map

// Function to compute covariances for two securities

let getCov xId yId (stockData: Map<string,StockData list>) =
    let xRet = 
        stockData[xId] 
        |> List.map (fun x -> x.Date,x.Return) 
        |> Map
    let yRet = 
        stockData[yId]
        |> List.map (fun y -> y.Date, y.Return)
        |> Map
    let overlappingDates =
        [ xRet.Keys |> set
          yRet.Keys |> set]
        |> Set.intersectMany
    [ for date in overlappingDates do xRet[date], yRet[date]]
    |> Seq.covOfPairs

let allTickers = 
    [ 
        "VTI" 
        "BND" 
        "Book leverage" 
    ]

let covariances portfolio=
    [ for rowTick in allTickers do 
        [ for colTick in allTickers do
            getCov rowTick colTick (stockData portfolio) ]]
    |> dsharp.tensor

// Function to compute the means

let means portfolio =
    [ for ticker in allTickers do 
        (stockData portfolio)[ticker]
        |> List.averageBy (fun x -> x.Return)]
    |> dsharp.tensor

// This will provide a type showing the average returns, variances, covariances and weights for the mean-variance optimal portfolio with the Long-Only/Long-Short + VTI + BND

type PortfolioStatistics = 
    { 
        Mean: Tensor
        Variance: Tensor
        Covariance: Tensor
        Weights: Tensor
    }

let PortfolioStatistics portfolio = 
    let stockData = stockData portfolio
    let cov = covariances portfolio
    let mean = means portfolio
    let w' = dsharp.solve(cov,mean)
    let w = w' / w'.sum()
    {Mean = dsharp.matmul(w, mean);
    Variance = w.matmul(cov).matmul(w);
    Covariance = cov; 
    Weights = w}

let LongOnlyStrategyStatistics = PortfolioStatistics longOnlyPortfolio
let LongShortStrategyStatistics = PortfolioStatistics longOnlyPortfolio

// Now we need a function that represent the members of the portfolio by symbol and weights

let weights portfolio =
    let w = (PortfolioStatistics portfolio).Weights
    Seq.zip allTickers (w.toArray1D<float>())
    |> Map.ofSeq

// Find the mutually overlapping time period

let stockDataByDate portfolio =
    (stockData portfolio).Values
    |> Seq.toList
    |> List.collect id 
    |> List.groupBy(fun x -> x.Date) 
    |> List.sortBy fst

let stockDataByDate6040 =
    stockData6040.Values
    |> Seq.toList
    |> List.collect id 
    |> List.groupBy(fun x -> x.Date) 
    |> List.sortBy fst

let allAssetsStart portfolio =
    stockDataByDate portfolio
    // find the first array element where there are as many stocks as you have symbols
    |> List.find(fun (month, stocks) -> stocks.Length = allTickers.Length)
    |> fst // convert (month, stocks) to month

let allAssetsEnd portfolio=
    stockDataByDate portfolio
    // find the last array element where there are as many stocks as you have symbols
    |> List.findBack(fun (month, stocks) -> stocks.Length = allTickers.Length)
    |> fst // convert (month, stocks) to month

let allAssetsStart6040 =
    stockDataByDate6040
    // find the first array element where there are as many stocks as you have symbols
    |> List.find(fun (month, stocks) -> stocks.Length = tickers.Length)
    |> fst // convert (month, stocks) to month

let allAssetsEnd6040=
    stockDataByDate6040
    // find the last array element where there are as many stocks as you have symbols
    |> List.findBack(fun (month, stocks) -> stocks.Length = tickers.Length)
    |> fst // convert (month, stocks) to month

let stockDataByDateComplete portfolio =
    stockDataByDate portfolio
    |> List.filter(fun (date, stocks) -> 
        date >= (allAssetsStart portfolio) &&
        date <= (allAssetsEnd portfolio))

let stockDataByDateComplete6040 =
    stockDataByDate6040
    |> List.filter(fun (date, stocks) -> 
        date >= allAssetsStart6040 &&
        date <= allAssetsEnd6040)

// Fuction to compute the monthly returns

let portfolioMonthReturn weights monthData =
    weights
    |> Map.toList
    |> List.map(fun (symbol, weight) ->
        let symbolData = 
            match monthData |> List.tryFind(fun x -> x.Symbol = symbol) with
            | None -> failwith $"You tried to find {symbol} in the data but it was not there"
            | Some data -> data
        symbolData.Return*weight)
    |> List.sum    

// Function to obtain the returns of the MVE portfolio

let portMVE portfolio=
    stockDataByDateComplete portfolio
    |> List.map(fun (date, data) -> 
        { Symbol = "MVE"
          Date = date
          Return = portfolioMonthReturn (portfolio |> weights) data })

// Provides a weight of 60% to the VTI and a weight of 40% to the BND

let weights6040 = Map [("VTI",0.6);("BND",0.4)]

// Function to obtain the returns of the 60/40 portfolio

let port6040 = 
    stockDataByDateComplete6040
    |> List.map(fun (date, data) -> 
        { Symbol = "60/40"
          Date = date 
          Return = portfolioMonthReturn weights6040 data} )

// MVE portfolio using long-only + other assets

let longOnlyStrategyMVE = portMVE longOnlyPortfolio

// MVE portfolio using long-short + other assets

let longShortStrategyMVE = portMVE longShortPortfolio

// Functions to compute and plot cumulative returns

let cumulateReturns xs =
    let folder prev current =
        let newReturn = prev.Return * (1.0+current.Return)
        { current with Return = newReturn}
    
    match xs |> List.sortBy (fun x -> x.Date) with
    | [] -> []
    | h::t ->
        ({ h with Return = 1.0+h.Return}, t) 
        ||> List.scan folder

let longOnlyMVECumulative = 
    longOnlyStrategyMVE 
    |> cumulateReturns

let longShortMVECumulative = 
    longShortStrategyMVE 
    |> cumulateReturns

let port6040Cumulative = 
    port6040
    |> cumulateReturns

let longOnlyChartMVE = 
    longOnlyMVECumulative
    |> List.map(fun x -> x.Date, x.Return)
    |> Chart.Line
    |> Chart.withTraceInfo(Name="MVE Long-only")

let longShortChartMVE = 
    longShortMVECumulative
    |> List.map(fun x -> x.Date, x.Return)
    |> Chart.Line
    |> Chart.withTraceInfo(Name="MVE Long-short")

let chart6040 = 
    port6040Cumulative
    |> List.map(fun x -> x.Date, x.Return)
    |> Chart.Line
    |> Chart.withTraceInfo(Name="60/40")

let chartCombined =
    [longOnlyChartMVE; longShortChartMVE;  chart6040]
    |> Chart.combine
    |> Chart.withTitle("Cumulative returns (without volatility management)")
    |> Chart.withSize (1000.0, 700.0)
Chart.show(chartCombined)

// Function to lever the portfolios based on the 10% annualized target volatility 

let normalize10pctVol xs =
    let vol = xs |> List.map(fun x -> x.Return) |> Seq.stDev
    let annualizedVol = vol * sqrt(12.0)
    xs 
    |> List.map(fun x -> { x with Return = x.Return * (0.1/annualizedVol)})

// Cumulative returns of the normalized volatility returns

let longOnlyMVECumulativeNormlizedVol = 
    longOnlyStrategyMVE
    |> normalize10pctVol 
    |> cumulateReturns

let longShortMVECumulativeNormlizedVol = 
    longShortStrategyMVE
    |> normalize10pctVol 
    |> cumulateReturns

let port6040CumulativeNormlizedVol = 
    port6040
    |> normalize10pctVol 
    |> cumulateReturns

let longOnlyChartMVENormlizedVol = 
    longOnlyMVECumulativeNormlizedVol
    |> List.map(fun x -> x.Date, x.Return)
    |> Chart.Line
    |> Chart.withTraceInfo(Name="Long-only (10%)")

let longShortChartMVENormlizedVol = 
    longShortMVECumulativeNormlizedVol
    |> List.map(fun x -> x.Date, x.Return)
    |> Chart.Line
    |> Chart.withTraceInfo(Name="Long-short (10%)")

let chart6040NormlizedVol = 
    port6040CumulativeNormlizedVol
    |> List.map(fun x -> x.Date, x.Return)
    |> Chart.Line
    |> Chart.withTraceInfo(Name="60/40")

let chartCombinedNormlizedVol =
    [longOnlyChartMVENormlizedVol; longShortChartMVENormlizedVol ; chart6040NormlizedVol ]
    |> Chart.combine
    |> Chart.withTitle("Cumulative returns (target volatility of 10%)")
    |> Chart.withSize (1000.0, 700.0)
Chart.show(chartCombinedNormlizedVol)

// Annual returns and Sharpe ratios of the 3 portfolios

let annualReturns portfolio (year:float) =
    portfolio
    |> List.map (fun x -> x.Return)
    |> List.average
    |> fun avg -> avg*year

let annualStd portfolio (year:float) =
    portfolio 
    |> List.map (fun x -> x.Return)
    |> Seq.stDev
    |> fun std -> std * sqrt(year)

// Returns and Sharpe ratios without volatility management

let annualReturnLongOnlyMve = round 4 (annualReturns longOnlyStrategyMVE 12.0)
let sharpeLongOnlyMve = round 2 (annualReturnLongOnlyMve/(annualStd longOnlyStrategyMVE 12.0))

let annualReturnLongShortMve = round 4 (annualReturns longShortStrategyMVE 12.0)
let sharpeLongShortMve = round 2 (annualReturnLongShortMve/(annualStd longShortStrategyMVE 12.0))

let annualReturns6040 = round 4 (annualReturns port6040 12.0)
let sharpe6040 = round 2 (annualReturns6040/(annualStd port6040 12.0))

// Returns and Sharpe ratios with target annualized volatility of 10%

let annualReturnLongOnlyMveManaged = round 4 (annualReturns (longOnlyStrategyMVE |> normalize10pctVol) 12.0)
let sharpeLongOnlyMveManaged = round 2 (annualReturnLongOnlyMveManaged/(annualStd (longOnlyStrategyMVE |> normalize10pctVol) 12.0))

let annualReturnLongShortMveManaged = round 4 (annualReturns (longShortStrategyMVE |> normalize10pctVol) 12.0)
let sharpeLongShortMveManaged = round 2 (annualReturnLongShortMveManaged/(annualStd (longShortStrategyMVE |> normalize10pctVol) 12.0))

let annualReturns6040Managed = round 4 (annualReturns (port6040 |> normalize10pctVol) 12.0)
let sharpe6040Managed = round 2 (annualReturns6040Managed/(annualStd (port6040 |> normalize10pctVol) 12.0))

// Build a table with the average annualized returns and Sharpe ratios

let header2 = ["<b>Strategy<b>";"<b>Average Annualized Excess Return (%)<b>";"<b>Annualized Sharpe Ratio<b>"]
let rows2 = 
    [
        ["MVE long-only"; (annualReturnLongOnlyMve * 100.0).ToDisplayString();sharpeLongOnlyMve.ToDisplayString()]                
        ["MVE long-short"; (annualReturnLongShortMve * 100.0).ToDisplayString();sharpeLongShortMve.ToDisplayString()]        
        ["60/40"; (annualReturns6040 * 100.0).ToDisplayString();sharpe6040.ToDisplayString()]
        ["MVE long-only (10%)"; (annualReturnLongOnlyMveManaged * 100.0).ToDisplayString();sharpeLongOnlyMveManaged.ToDisplayString()]                
        ["MVE long-short (10%)"; (annualReturnLongShortMveManaged * 100.0).ToDisplayString();sharpeLongShortMveManaged.ToDisplayString()]        
        ["60/40 (10%)"; (annualReturns6040Managed * 100.0).ToDisplayString();sharpe6040Managed.ToDisplayString()]     
    ]

// Requested table

let table2 = Chart.Table(header2, rows2) |> Chart.withSize (900.0, 500.0) |> Chart.withTitle("Average Annualized Returns and Sharpe Ratios")
table2

<!-- Plotly chart will be drawn inside this DIV -->